<a href="https://colab.research.google.com/github/sudhirshahu51/RAG/blob/main/pdf_query_with_chromadb_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install llama-index langchain chromadb pypdf llama-index-llms-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:0

# 1. **Text Extraction from PDF file**

In [18]:
from pypdf import PdfReader

reader = PdfReader('/content/drive/MyDrive/Colab Notebooks/Data/microsoft_annual_report_2022.pdf')
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]
len(pdf_texts)

90

In [19]:
#for text in pdf_texts[:5]:
print(pdf_texts[0])

1 
Dear shareholders, colleagues, customers, and partners:  
We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like 
the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is 
ongoing. At the same time, we are entering a technological era with the potential to power awesome advancements 
across every sector of our economy and society. As the world’s largest software company, this places us at a historic 
intersection of opportunity and responsibility to the world around us.  
Our mission to empower every person and every organization on the planet to achieve more has never been more 
urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every 
industry are increasingly looking to digital technology to overcome today’s challenges and emerge stronger. And no 
company is better positioned to help t

# 2. **Splitting Chunks**

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter #understand sentence

In [22]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

#print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")


Total chunks: 344


In [23]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

#print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Total chunks: 349


# **3. Create embeddings**

In [24]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


In [25]:
embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[array([ 4.25627008e-02,  3.32118124e-02,  3.03400941e-02, -3.48665677e-02,
        6.84165210e-02, -8.09090883e-02, -1.54743837e-02, -1.45092339e-03,
       -1.67444516e-02,  6.77077174e-02, -5.05413823e-02, -4.91953641e-02,
        5.13999313e-02,  9.19273049e-02, -7.17783868e-02,  3.95196825e-02,
       -1.28335450e-02, -2.49474961e-02, -4.62286212e-02, -2.43575163e-02,
        3.39496508e-02,  2.55024210e-02,  2.73171309e-02, -4.12621116e-03,
       -3.63383405e-02,  3.69089260e-03, -2.74304301e-02,  4.79671173e-03,
       -2.88962368e-02, -1.88706759e-02,  3.66662778e-02,  2.56958883e-02,
        3.13128494e-02, -6.39343634e-02,  5.39440550e-02,  8.22534934e-02,
       -4.17568274e-02, -6.99579250e-03, -2.34860424e-02, -3.07479482e-02,
       -2.97919312e-03, -7.79093876e-02,  9.35312267e-03,  3.16284667e-03,
       -2.22570375e-02, -1.82946529e-02, -9.61245876e-03, -3.15068923e-02,
       -5.51966717e-03, -3.27030383e-02,  1.68029770e-01, -4.74596955e-02,
       -5.00168316e-02, 

In [26]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

349

In [27]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

In [30]:
import textwrap

In [31]:
for document in retrieved_documents:
    print(textwrap.fill(document))
    print('\n')

74 note 13 — unearned revenue unearned revenue by segment was as
follows : ( in millions ) june 30, 2022 2021 productivity and business
processes $ 24, 558 $ 22, 120 intelligent cloud 19, 371 17, 710 more
personal computing 4, 479 4, 311 total $ 48, 408 $ 44, 141 changes in
unearned revenue were as follows : ( in millions ) year ended june 30,
2022 balance, beginning of period $ 44, 141 deferral of revenue 110,
455 recognition of unearned revenue ( 106, 188 ) balance, end of
period $ 48, 408 revenue allocated to remaining performance
obligations, which includes unearned revenue and amounts that will be
invoiced and recognized as revenue in future periods, was $ 193
billion as of june 30, 2022, of which $ 189 billion is related to the
commercial portion of revenue. we expect to recognize approximately 45
% of this revenue over the next 12 months and the remainder
thereafter. note 14 — leases


that are not sold separately. • we tested the mathematical accuracy of
management ’ s calculat

# 4. Calling LLM using GROQ API

In [44]:
from google.colab import userdata
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
groq_key = userdata.get('GROQ_API')
llm = Groq(model="llama3-70b-8192", api_key=groq_key) #https://docs.llamaindex.ai/en/stable/examples/llm/groq/#streaming

In [37]:
response = llm.complete("Explain the importance of low latency LLMs")

In [40]:
print(response)

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Interactive Systems**: In interactive systems like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, ensuring a seamless user experience. Delays in response can lead to user frustration and abandonment.
2. **Real-time Decision Making**: In applications like autonomous vehicles, medical diagnosis, or financial trading, low-latency LLMs can quickly process and analyze vast amounts of data to make timely decisions, which can be critical in life-or-death situations or high-stakes environments.
3. **Live Streaming and Broadcasting**: Low-latency LLMs can facilitate real-time language translation, subtitling, or closed captions for live events, such as conferences, sports, or news broadcasts, making them more accessible to a broader audience.
4. *

In [41]:
response = llm.stream_complete("Explain the importance of low latency LLMs")

In [42]:
for r in response:
    print(r.delta, end="")

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Conversational AI**: In conversational AI, such as chatbots, voice assistants, and dialogue systems, low-latency LLMs enable rapid response times, making interactions feel more natural and human-like. This is particularly important in applications like customer service, where timely responses can significantly impact user experience and satisfaction.
2. **Real-time Language Translation**: Low-latency LLMs can facilitate real-time language translation, enabling instant communication across language barriers. This is critical in scenarios like international business meetings, diplomatic negotiations, or emergency response situations.
3. **Speech Recognition**: Fast and accurate speech recognition relies on low-latency LLMs. This is essential in applications like voice-controlled devices, voi

In [46]:
def rag(query, retrieved_documents, llm):
    information = "\n\n".join(retrieved_documents)

    messages = [
    ChatMessage(
        role="system", content="You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report. You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
    ),
    ChatMessage(role="user", content=f"Question: {query}. \n Information: {information}"),
    ]

    resp = llm.stream_chat(messages)
    return resp


In [49]:
output = rag(query=query, retrieved_documents=retrieved_documents, llm = llm)

for r in output:
    print(r.delta, end="")

The total revenue was $198,270 million.